### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_movies = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [ ]:
response =requests.get(query_movies)
response

In [ ]:
# Create an empty list to store the reviews
reviews_list = []
#movies_per_page = 10

# loop through pages 0-19
for page in range(0,20):
    # create query with a page number
    #start_index = page * movies_per_page
    #query_movies =f"{url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}&end_date={end_date}" 
    query_movies = f"{query_movies}&page={str(page)}"
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    results = requests.get(query_movies).json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    # Try and save the reviews to the reviews_list
    try:
      # loop through the reviews["response"]["docs"] and append each review to the list
      for doc in results["response"]["docs"]:
            reviews_list.append(doc)
               
         
      # Print the page that was just retrieved
      print(f"Page {page} retrieved")
    except:
     # Print the page number that had no results then break from the loop
      print(f"No page {page} found")
      break
     

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
# Print results in JSON format
print(json.dumps(reviews_list[:5], indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_df["title"] = reviews_df["headline.main"].apply(
     lambda headline:headline[headline.find("\u2018")+1:headline.find("\u2019 Review")]
)

reviews_df


In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)

reviews_df

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title = reviews_df["title"].to_list()
titles= title
titles

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url_tmdb = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []
# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1
#request_counter % 50 == 0
print ("application is sleeping")
time.sleep(3) 
# Loop through the titles
for title in titles:
    
    print(request_counter)
    if request_counter % 50 == 0:
        print(request_counter)
        time.sleep(3)
        print(f"Sleeping at {request_counter} requests")
    # Check if we need to sleep before making a request
    # Add 1 to the request counter
    request_counter += 1
    # Perform a "GET" request for The Movie Database
    data = requests.get(url_tmdb + title + tmdb_key_string).json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:    
     # Get movie id
        movie_id = data["results"][0]["id"]
        # Make a request for a the full movie details
        print(movie_id)
        # Execute "GET" request with url
        query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        response_data = requests.get(query_url).json()

        #print(response_data)
        # Extract the genre names into a list
        # Extract the spoken_languages' English name into a list
        # Extract the production_countries' name into a list
            
        #movie_details = response_data["results"][0] 
        genres = [genre["name"] for genre in response_data.get("genres")]
        print(genres)
        spoken_languages = [lang['english_name'] for lang in response_data.get('spoken_languages')]
        production_countries = [country['name'] for country in response_data.get('production_countries')]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_info = {
                    "title": response_data['title'],
                    "original_title": response_data['original_title'],
                    "budget": response_data['budget'],
                    "genre": genres,
                    "language": response_data['original_language'],
                    "spoken_languages": spoken_languages,
                    "homepage": response_data['homepage'],
                    "overview": response_data['overview'],
                    "popularity": response_data['popularity'],
                    "runtime": response_data['runtime'],
                    "revenue": response_data['revenue'],
                    "release_date": response_data['release_date'],
                    "vote_average": response_data['vote_average'],
                    "vote_count": response_data['vote_count'],
                    "production_countries": production_countries
                    }
        tmdb_movies_list.append(movie_info)
    except Exception as e: 
        # Print out the title that was found
        print(f"No page {title} found",e)


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

# Print results in JSON format
print(json.dumps(tmdb_movies_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df 

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df, reviews_df, how="inner", on="title")
merged_df

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genre", "spoken_languages", "production_countries"]
# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]   
# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype("str")
    # Loop through characters to remove
    for character in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(character, "", regex=False)

# Display the fixed DataFrame
merged_df


In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop("byline.person", axis=1)

In [ ]:
# Delete duplicate rows and reset index
merged_df =merged_df.drop_duplicates()

merged_df = merged_df.reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv("merged_movie_data.csv", index=False)